This notebook sets up a database and installs python modules to use with the crossword generator project. 



### Install the Weaviate client

In [ ]:
# Uncomment to clear your current pip cache
# !pip cache purge

# Uncomment to upgrade pip
# !pip install --upgrade pip

# Install client from public released
!pip3 install --no-cache -U "weaviate-client==4.*"

# Check installed client version
!pip show weaviate-client | grep Version

### Install additional Python libraries

In [ ]:
# # Import the Ollama python client
# !pip install ollama

# # Import spacy named entity recognition for puzzle generation
# #   It might take a few minutes to build spacy and setup the data
# !pip install spacy
# !python -m spacy download en_core_web_sm

### Install Ollama

Install an LLM and an embedding model.

Ollam should start after you install it. To check if Ollama is running, open localhost:11434 in a browser

In [ ]:
# # Uncomment to install Ollama
# !ollama pull llama3         # The LLM
# !ollama pull all-minilm     # For embeddings

### Connect the client to a local Weaviate instance


In [ ]:
import weaviate

client = weaviate.connect_to_local()

# # Uncomment to check the connection
# client.is_ready()

### Check if the Ollama module is enabled in Weaviate

If the Ollama modules are not configured, enable the `text2vec-ollama` module and the `generative-ollama` module in your Weaviate [configuration file](/developers/weaviate/installation#configuration-files).

In [ ]:
meta_info = client.get_meta()
if_ok = True

if 'text2vec-ollama' not in meta_info["modules"] :
    print("Enable the text2vec-ollama module.")
    if_ok = False

if 'generative-ollama' not in meta_info["modules"] :
    print("Enable the generative-ollama module.")
    if_ok = False

if if_ok:
    print("Modules are installed")

### Set the collection name

You will need a collection to store your data. This code lets you choose a collection name and cleans up any earlier versions if they exist.

In [ ]:
# Set the collection name
collection_name = "CrosswordPuzzles"

# # Uncomment to remove old versions of this collection
# if (client.collections.exists(collection_name)):
#     client.collections.delete(collection_name)
#     print(f"Removed old collection: {collection_name}")

### Define a collection

The local collection holds some books from [Project Gutenberg](https://www.gutenberg.org/).

This definition is very basic. When the books in the database are converted to vector embeddings below, they aren't given any meta-data to record as properties here.  

In [42]:

from weaviate.classes.config import Property, DataType, Configure

#  Define the collection
collection = client.collections.create(
    name=collection_name,
    description="Source texts for puzzles",
    properties=[
        Property(name="text", data_type=DataType.TEXT),
    ],
    vectorizer_config=Configure.Vectorizer.text2vec_ollama(
        # This assumes Weaviate is in a Docker container and Ollama is local
        api_endpoint="host.docker.internal:11434",
        model="all-minilm"
    ),
    generative_config=Configure.Generative.ollama(
        # This assumes Weaviate is in a Docker container and Ollama is local
        api_endpoint="host.docker.internal:11434",
        model="llama3"
    )
)

# # Uncomment to check the collection definition
# collection_definition = client.collections.export_config(collection_name)
# print(f"Name: {collection_definition.name}     Description: {collection_definition.description}")


### Import the data into the collection

Process some text files (books from Project Guttenberg) to use as project specific data. 

In [ ]:
import os
import spacy
import ollama


# Initiate spacy to process the files
nlp = spacy.load('en_core_web_sm')

# Get a list of the sources
source_dir = "../inputs/"
source_files = [f for f in os.listdir(source_dir) if os.path.isfile(source_dir + f)]

for sf in source_files:
    # Uncomment to show progress
    print("Starting new text")

    with open(source_dir + sf, 'r') as f:
        sentences = []
        header_flag = True

        # Uncomment to show progress
        print("Reading")

        source_text = f.read()

        # Split each source file into sentence-like strings
        for s in nlp(source_text).sents:
            s = str(s)
            counter = 0
            increment = 20   # When to bump the counters

            # Don't include file header information
            if header_flag:
                if s.startswith('*** START'):
                    header_flag = False
                continue
            else:
                if len(s) > 0:
                    sentences.append(s)

        # Uncomment to show progress
        print("Embedding vectors")

        # Create embeddings for the sentences
        with collection.batch.dynamic() as batch:
            for snt in sentences:
                response = ollama.embeddings(model="all-minilm", prompt=snt)
                embedding = response["embedding"]

                # Uncomment to show progress
                counter += 1
                if (counter % increment == 0):
                    print(f".", end=" ")
                if (counter % (increment * 10) == 0):
                    print("\n")

                # Batch add objects to the collection
                batch.add_object(
                    properties = {"text": snt},
                    vector = embedding,
                )
            # Uncomment to show progress
            print("\n")


# NOTE: The import process takes about seven minutes on my laptop.

### Check the data upload

In [46]:
# # Uncomment to print the number of objects
# collection = client.collections.get(collection_name)
# response = collection.aggregate.over_all(total_count=True)
# print(f"Collection size: {response.total_count}")

# Uncomment to print the first 3 objects
response = collection.query.fetch_objects(
    limit=3,
    include_vector=True
    )
for o in response.objects:
    print(o.properties)
    print(o.vector)

{'text': 'By a curious chance you came\nupon her tresses.'}
{'default': [-0.027634859085083008, 0.04151489958167076, 0.3347301781177521, 0.2736194431781769, 0.2591266334056854, -0.40904897451400757, 0.46258264780044556, -0.03974155709147453, 0.2031933069229126, 0.11512838304042816, 0.11533090472221375, -0.13739265501499176, 0.03271320462226868, -0.22720950841903687, -0.02363964170217514, 0.19207370281219482, 0.04356100410223007, -0.06620224565267563, -0.5538679957389832, 0.3055519461631775, -0.2001664936542511, 0.17468655109405518, 0.2430480271577835, -0.11881931871175766, -0.12238630652427673, -0.29435601830482483, 0.1900881826877594, 0.09005919843912125, 0.13840240240097046, -0.06750593334436417, -0.4508368968963623, 0.11923889815807343, -0.38669541478157043, 0.013195343315601349, 0.0752328634262085, 0.09057129919528961, 0.19002485275268555, -0.17427900433540344, 0.32527437806129456, 0.0414743572473526, -0.17271548509597778, -0.45650094747543335, 0.14941589534282684, 0.07613553106784